In [ ]:
import ipywidgets as widgets
import numpy as np
import bqplot.pyplot as bq
import matplotlib.pyplot as plt
import tempNcolor as tc

Here is a simulation containing two stars in a binary system with their center of mass marked with a white cross in the center. This interactive allows you to change the mass of each of the stars and observe how their relative postions change, along with their radius, as a result.

In [ ]:
## FUNCTIONS ##

def x1_x2_update_V2(m1,m2,x1,x2):
    '''
    Takes the masses, m1 and m2, and 1-D positions, x1, and x2, of 2 stars.
    Uses these values to compute the center of mass of these objects.
    Then, with the intention is that the center of mass is held constant (at (0,0)),
    it computes updated positions x1 and x2 and returns these.
    '''
    new_CM = (m1*x1+m2*x2)/(m1+m2)
    x1 -= new_CM
    x2 -= new_CM
    return [x1],[x2]

def Rad_calc(mass):
    '''
    Determines radius of a star given the mass.
    Uses approximate fit from ZAMS line
    '''
    return 10 ** (0.0757*(np.log10(mass))**4 - 
                  0.1348*(np.log10(mass))**3 - 
                  0.1355*(np.log10(mass))**2 + 
                  0.8546*np.log10(mass) - 0.0516)

def Temp_calc(mass):
    '''
    Determines the approximate temperature of a
    star given its mass. Uses a fit aquired from Eker et al (2015).
    '''
    return 10 ** (0.6287*np.log10(mass) + 3.7404)

def h(change=None):
    '''
    This function updates the x values of the centers of the stars, as well as the radii of the stars.
    It takes no arguments and sets up initial conditions for the x positions. 
    ##Updated##
    Now also handles the position of each star for output to the screen and the temperature of the stars.
    The temp is converted to color which is used to show the color of the stars in the figure.
    
    This function, along with the later widgetname.observe(h, names=['value']) allow the .value 
    commands to update each time the widget is adjusted without having to rerun the code. This
    makes function calls such as Rad_calc easier to implement.
    '''
    # intial x positions of each star.
    x1 = -10
    x2 = 10
    
    # updates the x position of each star as the slider is adjusted
    star1.x,star2.x = x1_x2_update_V2(star1_slider.value,star2_slider.value,x1,x2)
    
    # updates the radii of each star
    r1= 300*int(10*Rad_calc(star1_slider.value))
    r2 = 300*int(10*Rad_calc(star2_slider.value))
    star1.default_size = r1
    star2.default_size = r2
    
    # changes the value of the textbox that outputs the distance of each star from
    # the center of mass
    pos1,pos2 = star1.x,star2.x
    star1_output.value = '{:.2f}'.format(abs(pos1[0]))
    star2_output.value = '{:.2f}'.format(pos2[0])
    
    # Determines the approximate temperature of each star and updates
    # with mass. Then uses the code from tempNcolor to convert the temps
    # to hexidecimal color so that the color of each star is approximately
    # accurate.
    temp1 = Temp_calc(star1_slider.value)
    temp2 = Temp_calc(star2_slider.value)
    counts1 = tc.temp2rgb(temp1)
    hex_color1 = tc.rgb2hex(counts1)
    counts2 = tc.temp2rgb(temp2)
    hex_color2 = tc.rgb2hex(counts2)
    star1.colors = [hex_color1[0]]
    star2.colors = [hex_color2[0]]


In [ ]:
## INTERACTIVE/DISPLAY WIDGETS ##

# Creates sliders for the mass of star 1 and star 2 respectively
# The initial mass of each star is set to 5.0 solar masses
# The limits are such that each star varies from 0.1 to 24.0 solar masses
# in 0.1 solar mass increments. This allows (roughly) the full range of 
# different star colors to be achieved.
star1_slider = widgets.FloatSlider(
    value=5.0,
    min=0.1,
    max=24.05,
    step=0.1,
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
star2_slider = widgets.FloatSlider(
    value=5.0,
    min=0.1,
    max=24.05,
    step=0.1,
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

# Creates textbox widgets to display the distances of each star from the center of mass.
# These are noninteactable so that students may only read the output.
star1_output = widgets.Text(
    value = str(10),
    style = {'description_width': 'initial'},
    description = 'Star 1 Distance from CM',
    disabled = True   
)

star2_output = widgets.Text(
    value = str(10),
    style = {'description_width': 'initial'},
    description = 'Star 2 Distance from CM',
    disabled = True   
)

In [ ]:
## PLOT/FIGURE ##

# Sets axis scale for x and y to 
sc_x = bq.LinearScale(min=-22,max=22)
sc_y = bq.LinearScale(min=-10,max=10)

# Sets up the axes, grid-lines are set to black so that they blend in with the background.
ax_x = bq.Axis(scale=sc_x, grid_color='black')
ax_y = bq.Axis(scale=sc_y, num_ticks=0, orientation='vertical', grid_color='black')

# Creates each star and the center of mass as points. The stars are circles and
# the center of mass is a cross. 
# Note: the initial x positions and colors of each star are just placeholders.
# They are immediately adjusted by the function h when the code is run.
star1 = bq.Scatter(x=[-10], y=[0], scales={'x': sc_x, 'y': sc_y}, colors=['black'], labels=['Star 1'])
star2 = bq.Scatter(x=[10], y=[0], scales={'x': sc_x, 'y': sc_y}, colors=['black'], labels=['Star 2'])
CM = bq.Scatter(x=[0], y=[0], scales={'x': sc_x, 'y': sc_y}, colors=['white'], 
                marker="cross", default_size=20, labels=['Center of Mass'])

# Labels the stars and CM on the plot
label = bq.Label(x=(-20,15,-3.8), y=[8,8,-10], scales={'x': sc_x, 'y': sc_y},
                   text=['Star 1', 'Star 2', 'Center of Mass'], default_size=15, font_weight='bolder',
                   colors=['white'], update_on_move=False)

# Makes the function h respond to changes in the mass values for each star.
star1_slider.observe(h, names=['value'])
star2_slider.observe(h, names=['value'])
# These make h respond to changes in each star's distance from the CM.
star1_output.observe(h, names=['value'])
star2_output.observe(h, names=['value'])
# Calls the function h to update the display in real time.
h()

# Creates the figure. The background color is set to black so that it looks like 'space.' Also,
# removes the default y padding.
fig = bq.Figure(title='Center of Mass Interactive', marks=[label,star1,star2,CM], axes=[ax_x, ax_y], 
                padding_y=0, animation=100, background_style={'fill' : 'black'})
# These ensure that the figure is as large as possible in its environment.
fig.layout.width = '100%'
fig.layout.height = "100%"

In [ ]:
## SCREEN DISPLAY ##

# Creates two vertical boxes, each containing a lable and mass slider for its corresponding star.
star1_box = widgets.VBox([widgets.Label('Mass of Star 1 (Solar Masses)'), star1_slider])
star2_box = widgets.VBox([widgets.Label('Mass of Star 2 (Solar Masses)'), star2_slider])
# Creates a box for the output of each star's distance from the CM.
output = widgets.VBox([star1_output,star2_output])

# Places the figure, sliders, and output into a Vbox. The figure is 
# alone in the top, while the sliders and output are in a Hbox
# inside the bottom of the Vbox.
top_box = fig
bottom_box = widgets.HBox([star1_box, star2_box, output])
BOX = widgets.VBox([top_box, bottom_box])
# Sets the dimensions of the box. Sets the entire width and the height of 
# just the top.
BOX.layout.width = '950px'
BOX.children[0].layout.height = '500px'
# Displays everything to the screen.
BOX